In [1]:
with open('input.txt') as file:
  lines = []
  for line in file:
    lines.append(line.strip())
print(f'line count: {len(lines)}')

line count: 188


In [2]:
test_text = '''seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4'''

test_lines = test_text.split('\n')

In [3]:
import re

#lines = test_lines[:]

seeds = list(map(int, re.findall(r'(\d+)', lines[0])))
transform = {}

def apply_transform(seeds, transform):
    print(f'seeds: {seeds}')
    print(f'transforms: {transform}')
    for i in range(len(seeds)):
        for ((src_min, src_max), dest) in transform.items():
            if src_min <= seeds[i] <= src_max:
                before_seed = seeds[i]
                seeds[i] += dest
                #print(f'changed seed[{i}]: {before_seed} -> {seeds[i]}')
                break

src = dest = ''
for line in lines[1:]:
    if line == '':
        apply_transform(seeds, transform)
        #print(f'{seeds}')
    elif 'map' in line:
        match = re.match(r'(?P<src>\w+)-to-(?P<dest>\w+) map:', line)
        (src, dest) = match.groups()
        transform = {}
    else:
        [dest_range, src_range, len_range] = list(map(int, re.findall(r'(\d+)', line)))
        transform[(src_range, src_range+len_range-1)] = dest_range - src_range

apply_transform(seeds, transform)

print(f'locations: {seeds}')
print(f'The lowest location number that corresponds to any of the initial seed numbers is: {min(seeds)}')

seeds: [2494933545, 159314859, 4045092792, 172620202, 928898138, 554061882, 2740120981, 81327018, 2031777983, 63513119, 2871914181, 270575980, 2200250633, 216481794, 3289604059, 25147787, 3472625834, 10030240, 260990830, 232636388]
transforms: {}
seeds: [2494933545, 159314859, 4045092792, 172620202, 928898138, 554061882, 2740120981, 81327018, 2031777983, 63513119, 2871914181, 270575980, 2200250633, 216481794, 3289604059, 25147787, 3472625834, 10030240, 260990830, 232636388]
transforms: {(2724782980, 3747465992): 547501303, (4195038636, 4294967295): -4056851145, (797621236, 925606014): 1562002523, (2224466386, 2490932641): -1562014457, (714355413, 797621235): 214562772, (3891516474, 4195038635): -2879332466, (1098322140, 1306829962): 1965454320, (1306829963, 1357355654): 887408203, (2091837170, 2224466385): -1734730582, (2490932642, 2605792542): -246168784, (3747465993, 3891516473): -1697278308, (925606015, 1098322139): -435870211, (138187491, 714355412): 2349421047, (2605792543, 272478

In [4]:
import re

#lines = test_lines[:]
seeds = []
transform = {}

line1 = re.findall(r'(\d+) (\d+)', lines[0])
for (start, num) in line1:
    seeds.append((int(start), int(start)+int(num)-1))

def apply_transform(seeds, transform):
    print(f'seeds: {seeds}')
    print(f'transforms: {transform}')
    new_seeds = []
    while len(seeds) > 0:
        (low, high) = seeds.pop()
        for ((src_min, src_max), dest) in transform.items():
            if src_min <= low <= high <= src_max:
                new_seeds.append((low+dest, high+dest))
                #print(f'changed seed[({low}-{high})] -> ({low+dest},{high+dest})')
                break
            if low <= src_min <= src_max <= high:
                new_seeds.append((src_min+dest, src_max+dest))
                #print(f'split changed seed[({src_min}-{high})] -> ({src_min+dest},{high+dest})')
                seeds.append((low, src_min-1))
                seeds.append((src_max+1, high))
                break
            if src_min <= low <= src_max <= high:
                new_seeds.append((low+dest, src_max+dest))
                #print(f'low half changed seed[({low}-{src_max})] -> ({low+dest},{src_max+dest})')
                low = src_max+1
            elif low <= src_min <= high <= src_max:
                new_seeds.append((src_min+dest, high+dest))
                #print(f'high half changed seed[({src_min}-{high})] -> ({src_min+dest},{high+dest})')
                high = src_min-1
        else: 
            #print(f'unchanged seed[({low},{high})]')
            new_seeds.append((low, high))
    return new_seeds

src = dest = ''
for line in lines[1:]:
    if line == '':
        seeds = apply_transform(seeds, transform)
        #print(f'{seeds}')
    elif 'map' in line:
        match = re.match(r'(?P<src>\w+)-to-(?P<dest>\w+) map:', line)
        (src, dest) = match.groups()
        transform = {}
    else:
        [dest_range, src_range, len_range] = list(map(int, re.findall(r'(\d+)', line)))
        transform[(src_range, src_range+len_range-1)] = dest_range - src_range

seeds = apply_transform(seeds, transform)

print(f'locations: {seeds}')
print(f'The lowest location number that corresponds to any of the initial seed numbers is: {min(map(lambda k: k[0], seeds))}')


seeds: [(2494933545, 2654248403), (4045092792, 4217712993), (928898138, 1482960019), (2740120981, 2821447998), (2031777983, 2095291101), (2871914181, 3142490160), (2200250633, 2416732426), (3289604059, 3314751845), (3472625834, 3482656073), (260990830, 493627217)]
transforms: {}
seeds: [(260990830, 493627217), (3472625834, 3482656073), (3289604059, 3314751845), (2200250633, 2416732426), (2871914181, 3142490160), (2031777983, 2095291101), (2740120981, 2821447998), (928898138, 1482960019), (4045092792, 4217712993), (2494933545, 2654248403)]
transforms: {(2724782980, 3747465992): 547501303, (4195038636, 4294967295): -4056851145, (797621236, 925606014): 1562002523, (2224466386, 2490932641): -1562014457, (714355413, 797621235): 214562772, (3891516474, 4195038635): -2879332466, (1098322140, 1306829962): 1965454320, (1306829963, 1357355654): 887408203, (2091837170, 2224466385): -1734730582, (2490932642, 2605792542): -246168784, (3747465993, 3891516473): -1697278308, (925606015, 1098322139): -